In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
test = pd.read_csv('../datasets/Bot-IoT/Bot_test.csv')

## DETECTION

In [3]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') |  (data['proto'] =='igmp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['attack']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','saddr','sport','daddr','dport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    data_features = data_features.fillna(value=-1)
    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(".")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(":")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(".")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(":")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dport'] = data_features['dport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dport'] = data_features['dport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dport'] = data_features['dport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['saddr'] = data_features['saddr'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['daddr'] = data_features['daddr'].astype(int)
    data_features['dport'] = data_features['dport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['attack']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)


    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [4]:
xtest, ytest = preprocessing(test)

In [7]:
def evaluate_detection(model, x, y): 
    # Need to return FNR, F1, Precision, Recall, Accuracy, number of false positives, number of false negatives, number of true positives, number of true negatives
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y, axis=1)
    report = classification_report(y_true, y_pred, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    accuracy = report['accuracy']
    FNR = fn / (fn + tp)

    
    return FNR, f1, precision, recall, accuracy, cm

In [6]:
# Load models 
model35 = models.load_model('../code/models/Bot_pred5_3_5.hdf5')
model55 = models.load_model('../code/models/Bot_pred5_5_5.hdf5')
model105 = models.load_model('../code/models/Bot_pred5_10_5.hdf5')
model155 = models.load_model('../code/models/Bot_pred5_15_5.hdf5')
model510 = models.load_model('../code/models/Bot_pred5_5_10.hdf5')
model515 = models.load_model('../code/models/Bot_pred5_5_15.hdf5')
model1010 = models.load_model('../code/models/Bot_pred5_10_10.hdf5')
model1515 = models.load_model('../code/models/Bot_pred5_15_15.hdf5')

In [8]:
evaluate_detection(model35, xtest, ytest)

(0.007062733693394266,
 0.9961022861186998,
 0.9961049120986155,
 0.996100604406317,
 0.996100604406317,
 array([[7828,   23],
        [  17, 2390]], dtype=int64))

In [9]:
evaluate_detection(model55, xtest, ytest)

(0.002077274615704196,
 0.9977599378678729,
 0.9977664653229681,
 0.9977578475336323,
 0.9977578475336323,
 array([[7833,   18],
        [   5, 2402]], dtype=int64))

In [10]:
evaluate_detection(model105, xtest, ytest)

(0.0008309098462816784,
 0.998538880937269,
 0.9985432183332518,
 0.9985377266523688,
 0.9985377266523688,
 array([[7838,   13],
        [   2, 2405]], dtype=int64))

In [11]:
evaluate_detection(model155, xtest, ytest)

(0.0008309098462816784,
 0.9742355937693422,
 0.9763511697337562,
 0.973776564632482,
 0.973776564632482,
 array([[7584,  267],
        [   2, 2405]], dtype=int64))

In [12]:
evaluate_detection(model510, xtest, ytest)

(0.0008309098462816784,
 0.9990255715296751,
 0.9990269411789433,
 0.9990251511015793,
 0.9990251511015793,
 array([[7843,    8],
        [   2, 2405]], dtype=int64))

In [13]:
evaluate_detection(model515, xtest, ytest)

(0.0004154549231408392,
 0.9747098208615351,
 0.9767774065443906,
 0.9742639890816923,
 0.9742639890816923,
 array([[7588,  263],
        [   1, 2406]], dtype=int64))

In [14]:
evaluate_detection(model1010, xtest, ytest)

(0.0016618196925633569,
 0.9745119462689669,
 0.9765284108073905,
 0.9740690193020082,
 0.9740690193020082,
 array([[7589,  262],
        [   4, 2403]], dtype=int64))

In [15]:
evaluate_detection(model1515, xtest, ytest)

(0.0008309098462816784,
 0.9747067054324294,
 0.9767472514233246,
 0.9742639890816923,
 0.9742639890816923,
 array([[7589,  262],
        [   2, 2405]], dtype=int64))

## CLASSIFICATION

In [16]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') |  (data['proto'] =='igmp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','saddr','sport','daddr','dport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    data_features = data_features.fillna(value=-1)
    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(".")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: x.split(":")[-1])
    data_features['saddr'] = data_features['saddr'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(".")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: x.split(":")[-1])
    data_features['daddr'] = data_features['daddr'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dport'] = data_features['dport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dport'] = data_features['dport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dport'] = data_features['dport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['saddr'] = data_features['saddr'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['daddr'] = data_features['daddr'].astype(int)
    data_features['dport'] = data_features['dport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)


    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'DoS', data['category'].replace('DoS', 1).replace(['Reconnaissance', 'DDoS', 'Theft'], 0))
    data_labels.insert(1, 'Reconnaissance', data['category'].replace('Reconnaissance', 1).replace([ 'DoS', 'DDoS', 'Theft'], 0))
    data_labels.insert(2, 'DDoS', data['category'].replace('DDoS', 1).replace(['DoS', 'Reconnaissance', 'Theft'], 0))
    data_labels.insert(3, 'Theft', data['category'].replace('Theft', 1).replace(['DoS', 'Reconnaissance', 'DDoS'], 0))
 
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)


    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [32]:
test = test[test['category'] != 'Normal']

In [33]:
xtest, ytest = preprocessing(test)

In [18]:
# Evaluate models that predict category: FNR, F1, Precision, Recall, Accuracy, FP, FN, TP, TN
def evaluate_cat(model, x, y): 
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y, axis=1)

    report = classification_report(y_true, y_pred, output_dict=True)
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    accuracy = report['accuracy']

    cat = set(y_pred)
    true_cat = set(y_true)

    conf_matrix = confusion_matrix(y_true, y_pred)
    return "f1: "+str('%.3f'%f1), "precision: "+str('%.3f'%precision), "recall: "+str('%.3f'%recall), "accuracy: "+str('%.3f'%accuracy), "true categories: "+str(true_cat), "predicted categories: "+str(cat), conf_matrix
    

In [19]:
# Load models 
model35 = models.load_model('../code/models/Bot_cat5_3_5.hdf5')
model55 = models.load_model('../code/models/Bot_cat5_5_5.hdf5')
model105 = models.load_model('../code/models/Bot_cat5_10_5.hdf5')
model155 = models.load_model('../code/models/Bot_cat5_15_5.hdf5')
model510 = models.load_model('../code/models/Bot_cat5_5_10.hdf5')
model515 = models.load_model('../code/models/Bot_cat5_5_15.hdf5')
model1010 = models.load_model('../code/models/Bot_cat5_10_10.hdf5')
model1515 = models.load_model('../code/models/Bot_cat5_15_15.hdf5')

In [34]:
evaluate_cat(model35, xtest, ytest)

('f1: 0.785',
 'precision: 0.809',
 'recall: 0.806',
 'accuracy: 0.806',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2, 3}',
 array([[2198,    0,  251,    0],
        [   0, 2523,    0,    2],
        [ 897,    0, 1596,    0],
        [   0,    0,  377,    7]], dtype=int64))

In [35]:
evaluate_cat(model55, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.802',
 'precision: 0.786',
 'recall: 0.823',
 'accuracy: 0.823',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[2212,    0,  237,    0],
        [   0, 2525,    0,    0],
        [ 771,    0, 1722,    0],
        [   0,    4,  380,    0]], dtype=int64))

In [36]:
evaluate_cat(model105, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.854',
 'precision: 0.837',
 'recall: 0.874',
 'accuracy: 0.874',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[2134,    0,  315,    0],
        [   0, 2525,    0,    0],
        [ 292,    0, 2201,    0],
        [   1,    2,  381,    0]], dtype=int64))

In [37]:
evaluate_cat(model155, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.867',
 'precision: 0.856',
 'recall: 0.887',
 'accuracy: 0.887',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[2060,    0,  389,    0],
        [   0, 2525,    0,    0],
        [ 116,    0, 2377,    0],
        [   0,    3,  381,    0]], dtype=int64))

In [38]:
evaluate_cat(model510, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.844',
 'precision: 0.825',
 'recall: 0.865',
 'accuracy: 0.865',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[2251,    0,  198,    0],
        [   0, 2525,    0,    0],
        [ 476,    0, 2017,    0],
        [   1,    2,  381,    0]], dtype=int64))

In [39]:
evaluate_cat(model515, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.860',
 'precision: 0.842',
 'recall: 0.881',
 'accuracy: 0.881',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[2196,    0,  253,    0],
        [   0, 2525,    0,    0],
        [ 301,    0, 2192,    0],
        [   0,    4,  380,    0]], dtype=int64))

In [40]:
evaluate_cat(model1010, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.869',
 'precision: 0.868',
 'recall: 0.888',
 'accuracy: 0.888',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[1960,    0,  489,    0],
        [   0, 2525,    0,    0],
        [   5,    0, 2488,    0],
        [   0,    1,  383,    0]], dtype=int64))

In [41]:
evaluate_cat(model1515, xtest, ytest)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('f1: 0.828',
 'precision: 0.848',
 'recall: 0.848',
 'accuracy: 0.848',
 'true categories: {0, 1, 2, 3}',
 'predicted categories: {0, 1, 2}',
 array([[1638,    0,  811,    0],
        [   0, 2525,    0,    0],
        [   0,    0, 2493,    0],
        [   0,    0,  384,    0]], dtype=int64))